In [1]:
import pymongo
from pymongo import MongoClient
from bson import SON

In [2]:
client = MongoClient()
db = client['gplaces']
placesColl = db['places']

places_geojson = db['places_geojson']
houses_geojson = db['houses_geojson']

In [50]:
import json
from bson.json_util import dumps

json_data = list()

for i,item in enumerate(houses_geojson.find()):
    item_dict = dict()
    item_dict['lng'] = item['loc']['coordinates'][0]
    item_dict['lat'] = item['loc']['coordinates'][1]
    
    #переделать на map reduce
    result_schools = places_geojson.aggregate([
       {
         #'$match': {"types": "school"},
         '$geoNear': {
            'near': { 'type': "Point", 'coordinates': [item_dict['lng'], item_dict['lat']] },
            'query': { 'types': ["library",'police'] },
            'distanceField': "dist",
            'maxDistance': 600,
            #'num': 1,
            'spherical': True
         }
       }
    ])   
    
    
    result = list(result_schools)
    
    if len(result)<=0:
        continue
        
    distance = sum([item for item in result])/len(result)
    #distance = 100*(1/(1+result[0]['dist'])) if len(result)>0 else 0#1/(1+result[0]['dist']) if len(result)>0 else 0.0000001
    
    #print(result)
    #if i>10:
    #    break
    
    item_dict['count'] = distance
    json_data.extend([item_dict])

with open('data_school.json', 'w') as fp:
    json.dump(json_data, fp)

KeyboardInterrupt: 

In [38]:
len([item for item in json_data if item['count']>0])

213